In [1]:
import csv as csv
import numpy as np
import pandas as pd
import sys
from sklearn import ensemble
import sklearn.preprocessing 
import sklearn.cross_validation

//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [375]:
print train.columns
print sum(train['TARGET'])
train.describe()

Index([u'ID', u'var3', u'var15', u'imp_ent_var16_ult1',
       u'imp_op_var39_comer_ult1', u'imp_op_var39_comer_ult3',
       u'imp_op_var40_comer_ult1', u'imp_op_var40_comer_ult3',
       u'imp_op_var40_efect_ult1', u'imp_op_var40_efect_ult3',
       ...
       u'saldo_medio_var33_hace2', u'saldo_medio_var33_hace3',
       u'saldo_medio_var33_ult1', u'saldo_medio_var33_ult3',
       u'saldo_medio_var44_hace2', u'saldo_medio_var44_hace3',
       u'saldo_medio_var44_ult1', u'saldo_medio_var44_ult3', u'var38',
       u'TARGET'],
      dtype='object', length=371)
3008


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


In [376]:
values = pd.DataFrame(train.TARGET.value_counts())
values['Percentage'] = 100 * values.TARGET / values.TARGET.sum()
values

,TARGET,Percentage
0,73012,96.043147
1,3008,3.956853


### Main program

In [492]:
# Split test / train data 
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    train[train.columns[:-1]],
    train[train.columns[-1]],
    test_size=0.5)

clf = sklearn.ensemble.GradientBoostingClassifier() 
clf.fit(X_train, y_train) 

y_pred = clf.predict(X_test)
print sklearn.metrics.roc_auc_score(y_pred, y_test)

0.600636994883


In [466]:
y_pred = clf.predict_proba(X_test)
y_pred = clf.predict(X_test)
print sklearn.metrics.roc_auc_score(y_pred, y_test)

0.52205721115


In [499]:
print sum(y_pred), sum(y_pred == 0)
y_pred = clf.predict_proba(X_test)
s = pd.Series(y_pred[:, 1]).rank(pct=1)
t = s.copy()
t[s > 1 - 1.0 * sum(y_test) / len(y_test)] = 1 
t[s <= 1 - 1.0 * sum(y_test) / len(y_test)] = 0
print 1.0*sum(t)/len(t)
print 1.0*sum(y_test)/len(y_test)
print sklearn.metrics.roc_auc_score(t, y_test)

56 37954
0.040726124704
0.0406998158379
0.596634746422


In [379]:
# Feature engineering
# Number of distinct values 
column_values = train.apply(lambda x: x.nunique(), axis=0) 

In [418]:
# Blacklist features 
blacklist_features = train.columns[
    (column_values == 1) &
    (train.columns != 'TARGET') & 
    (train.columns != 'ID')]
print blacklist_features.shape


(34,)


In [495]:
# Categorial features 
num_samples = train['ID'].count() 
print num_samples 
categorical_features = train.columns[
    (column_values <= 100) & 
    (column_values >= 3) & 
    (train.columns != 'TARGET') & 
    (train.columns != 'ID')]
print categorical_features.size
print sum(column_values[categorical_features]) 
print train[blacklist_features].shape
print train.shape

76020
165
3319
(76020, 34)
(76020, 371)


In [496]:
new_train = pd.DataFrame(
    train, 
    columns=train.columns[(train.columns != 'TARGET') & (train.columns != 'ID')]
)
for  i, ft in enumerate(categorical_features.tolist()):
    print i, 
    s = train[ft]
    t = pd.get_dummies(s, prefix=ft)
    new_train.drop([ft], axis=1)
    new_train[t.columns] = t 
    pass
print 
print new_train.shape 

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164
(76020, 3671)


In [447]:
new_train.shape

(76020, 536)

In [498]:
# Split test / train data 
training_data = new_train.copy() 
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    training_data,
    train['TARGET'],
    test_size=0.5,
    random_state = 2
)

clf = sklearn.ensemble.GradientBoostingClassifier(max_depth=4) 
clf.fit(X_train, y_train) 

y_pred = clf.predict(X_test) 
print sklearn.metrics.roc_auc_score(y_pred, y_test)

0.560095773831


In [ ]:
# Get the results 
print test.shape
new_test = pd.DataFrame(
    test, 
    columns=train.columns[(test.columns != 'TARGET') & (test.columns != 'ID')]
)
for  i, ft in enumerate(categorical_features.tolist()):
    print i, 
    s = train[ft]
    t = pd.get_dummies(s, prefix=ft)
    new_test.drop([ft], axis=1)
    new_test[t.columns] = t 
    pass
print 
print new_test.shape

In [ ]:
yy_test = clf.predict(new_test)

In [398]:
print yy_test, type(yy_test)
print sum(yy_test) 
print sum(yy_test == 0), sum(yy_test == 1)
print sum(y_pred == 0), sum(y_pred == 1)

[1 1 1 ..., 1 1 1] <type 'numpy.ndarray'>
74949
869 74949
7275 327
